In [1]:
from src.docstore import Docstore

from pathlib import Path
import random

import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F


/home/awehrs/miniconda3/envs/rar/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# docstore = Docstore.build(Path("data", "mock"))
# docstore.save(Path("tests", "test_artifacts", "test_docstore"))
docstore = Docstore.load(Path("..", "tests", "test_artifacts", "test_docstore"))


In [3]:
query_idx = random.choices(
    list(range(len(docstore._huggingface_dataset))),
    k=5,
)

In [4]:
queries = docstore._huggingface_dataset[query_idx]["Embeddings"]

In [5]:
queries = np.array(queries).astype(np.float32)

In [6]:
_, examples = docstore.get_nearest_examples(queries, 5)

In [7]:
n_neighbors = 7
search_date = "1/1/2022"
num_samples = 100
resample_freq = "M"
aggregation_fn = np.mean
coverage_ratio = .5
metadata_field = "Embeddings"

_dict = docstore.date_aligned_knn(queries, n_neighbors, search_date, num_samples, resample_freq, aggregation_fn, coverage_ratio, metadata_field)


DatetimeIndex(['2013-09-30', '2013-10-31', '2013-11-30', '2013-12-31',
               '2014-01-31', '2014-02-28', '2014-03-31', '2014-04-30',
               '2014-05-31', '2014-06-30', '2014-07-31', '2014-08-31',
               '2014-09-30', '2014-10-31', '2014-11-30', '2014-12-31',
               '2015-01-31', '2015-02-28', '2015-03-31', '2015-04-30',
               '2015-05-31', '2015-06-30', '2015-07-31', '2015-08-31',
               '2015-09-30', '2015-10-31', '2015-11-30', '2015-12-31',
               '2016-01-31', '2016-02-29', '2016-03-31', '2016-04-30',
               '2016-05-31', '2016-06-30', '2016-07-31', '2016-08-31',
               '2016-09-30', '2016-10-31', '2016-11-30', '2016-12-31',
               '2017-01-31', '2017-02-28', '2017-03-31', '2017-04-30',
               '2017-05-31', '2017-06-30', '2017-07-31', '2017-08-31',
               '2017-09-30', '2017-10-31', '2017-11-30', '2017-12-31',
               '2018-01-31', '2018-02-28', '2018-03-31', '2018-04-30',
      

RuntimeError: stack expects a non-empty TensorList

In [11]:
data = pd.concat([df for df in _dict["x"]])
data.columns = pd.to_datetime(data.columns, format="%Y-%m-%d")

In [13]:
index = pd.date_range(end="1/1/2022", freq="M", periods=120)

In [28]:
sorted(data.columns.tolist()) == data.columns.tolist()

True

In [45]:
y = data.truncate(before='1/1/1970', after='1/1/2010', axis=1)

In [48]:
window = y.columns[-1] - y.columns[0]
window

Timedelta('14245 days 00:00:00')

In [37]:
first = y.apply(pd.Series.first_valid_index, axis=1)

In [38]:
last = y.apply(pd.Series.last_valid_index, axis=1)

In [42]:
z = pd.concat([first, last], axis=1)

In [49]:
z["diff"] = z[1] - z[0]

In [50]:
z["coverage"] = z["diff"] / window

In [57]:
z["test"] = z["test"] >= .5

In [58]:
z

,0,1,diff,coverage,test
0,1990-12-31,2009-12-31,6940 days,0.487188,False
1,1989-12-31,2009-12-31,7305 days,0.512812,True
2,2004-12-31,2009-12-31,1826 days,0.128185,False
3,1993-12-31,2009-12-31,5844 days,0.410249,False
4,NaT,NaT,NaT,NaN,False
5,1970-12-31,2009-12-31,14245 days,1.000000,True
6,1993-12-31,2009-12-31,5844 days,0.410249,False
0,1997-12-31,2009-12-31,4383 days,0.307687,False
1,NaT,NaT,NaT,NaN,False
2,1990-12-31,2009-12-31,6940 days,0.487188,False


In [59]:
y[z["test"] == True]

,1970-12-31,1971-12-31,1972-12-31,1973-12-31,1974-12-31,1975-12-31,1976-12-31,1977-12-31,1978-12-31,1979-12-31,...,2000-12-31,2001-12-31,2002-12-31,2003-12-31,2004-12-31,2005-12-31,2006-12-31,2007-12-31,2008-12-31,2009-12-31
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.400000e+00,-1.400000e+00,-1.100000e+00,-2.200000e+00,-1.400000e+00,-0.300,6.000000e-01,1.000,-1.000000e+00,-6.300
5,2.000000e-02,2.000000e-02,0.020,2.000000e-02,3.000000e-02,3.000000e-02,3.000000e-02,4.000000e-02,5.000000e-02,6.000000e-02,...,4.200000e-01,4.600000e-01,4.800000e-01,4.700000e-01,5.000000e-01,0.540,6.400000e-01,0.670,7.400000e-01,0.720
4,7.056847e+03,7.335758e+03,7329.921,7.407367e+03,7.685857e+03,7.559144e+03,7.291840e+03,7.681017e+03,7.227564e+03,7.849364e+03,...,8.182686e+03,7.735488e+03,6.816722e+03,7.337773e+03,7.913726e+03,8522.522,9.112100e+03,9830.700,1.012510e+04,9428.500
5,3.707082e+03,3.855167e+03,4066.171,4.272766e+03,4.399818e+03,4.381208e+03,4.506737e+03,4.783021e+03,4.951762e+03,5.061095e+03,...,6.230468e+03,6.332833e+03,6.434386e+03,6.606158e+03,6.792108e+03,6954.036,7.351868e+03,7841.933,8.095314e+03,7911.799
6,3.490000e+01,2.210000e+01,163.400,5.081000e+02,3.759000e+02,3.407000e+02,1.743000e+02,6.350000e+01,3.030000e+01,3.890000e+01,...,4.500000e+00,2.600000e+00,2.800000e+00,1.100000e+00,2.400000e+00,3.700,2.600000e+00,7.800,7.100000e+00,-2.600
0,1.970000e+00,1.880000e+00,1.930,2.220000e+00,2.690000e+00,2.790000e+00,3.330000e+00,3.580000e+00,3.910000e+00,4.450000e+00,...,5.201000e+01,5.664000e+01,6.022000e+01,6.375000e+01,6.963000e+01,75.600,7.998000e+01,86.970,9.515000e+01,94.880
4,2.365514e+06,2.460610e+06,2470656.000,3.451090e+06,3.462392e+06,2.379365e+06,1.362219e+06,3.507918e+06,3.692057e+06,4.551328e+06,...,4.093810e+06,4.282126e+06,4.500514e+06,4.568022e+06,4.430981e+06,4522000.000,4.622948e+06,4885249.500,5.058186e+06,5754896.500
3,1.485100e+02,1.086700e+02,4.370,7.800000e+00,1.330000e+01,1.089000e+01,7.660000e+00,1.109000e+01,6.860000e+00,1.352000e+01,...,4.078750e+03,3.373210e+03,3.274600e+03,3.722580e+03,4.078330e+03,5610.910,6.683940e+03,6796.310,9.541450e+03,4887.570
5,3.707082e+03,3.855167e+03,4066.171,4.272766e+03,4.399818e+03,4.381208e+03,4.506737e+03,4.783021e+03,4.951762e+03,5.061095e+03,...,6.230468e+03,6.332833e+03,6.434386e+03,6.606158e+03,6.792108e+03,6954.036,7.351868e+03,7841.933,8.095314e+03,7911.799
3,6.342000e+02,6.062000e+02,598.800,5.357000e+02,5.493000e+02,5.859000e+02,5.911000e+02,5.927000e+02,5.788000e+02,4.460000e+02,...,4.625000e+02,4.973000e+02,5.193000e+02,5.733000e+02,7.380000e+02,835.200,8.120000e+02,811.000,8.089000e+02,816.000


In [40]:
len(_dict["data"][1].columns) - _dict["data"][4].loc[2].isnull().sum() < dropna_threshold

False

In [32]:
data_tensors = []
for df in _dict["data"]:
    data_tensors.append(torch.stack([torch.FloatTensor(x) for x in _dict["data"][2].values]))

torch.stack(data_tensors).shape

torch.Size([5, 7, 100])

In [13]:
len(_dict["data"][0].columns)

79

In [9]:
dfs = [pd.DataFrame(ex)[docstore.time_index] for ex in examples]

df = pd.concat(dfs)

In [10]:
df.columns = pd.to_datetime(df.columns, format="%Y-%m-%d")
df

,1960-12-31,1961-12-31,1962-12-31,1963-12-31,1964-12-31,1965-12-31,1966-12-31,1967-12-31,1968-12-31,1969-12-31,...,2011-12-31,2012-12-31,2013-12-31,2014-12-31,2015-12-31,2016-12-31,2017-12-31,2018-12-31,2019-12-31,2020-12-31
0,None,None,None,None,None,None,None,None,None,None,...,102.590,106.350,105.86,106.03,106.360,104.300,102.800,107.000,110.800,108.70
1,None,None,None,None,None,None,None,None,None,None,...,101.900,104.100,104.80,105.50,105.800,101.100,101.500,102.600,102.400,105.50
2,None,None,None,None,None,None,None,None,None,None,...,1.800,-1.800,4.00,0.90,0.700,0.700,6.500,-2.000,-5.400,0.90
3,None,None,None,None,None,None,None,None,None,None,...,109.200,113.900,111.10,103.60,99.500,108.800,113.300,118.300,118.200,115.70
4,None,None,None,None,None,None,None,None,None,None,...,134.900,140.900,141.40,143.60,146.700,155.300,158.400,160.000,163.000,165.20
0,None,None,None,None,None,None,None,None,None,None,...,5.500,5.100,9.01,10.40,9.272,15.036,16.351,13.619,14.526,15.20
1,None,None,None,None,None,None,None,None,None,None,...,0.600,-0.800,2.40,-4.10,1.400,1.900,0.500,1.100,0.000,0.80
2,None,None,None,None,None,None,None,None,None,None,...,3.500,3.750,NaN,4.75,4.250,NaN,5.000,NaN,NaN,6.25
3,None,None,None,None,None,None,None,None,None,None,...,10.000,11.000,9.80,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,None,None,None,None,None,None,None,None,None,None,...,NaN,7612.390,9552.16,9805.55,10743.010,NaN,NaN,NaN,NaN,NaN


In [55]:
df_resample = df.resample("M", axis=1).agg("mean").ffill(axis=1)
df_resample

,1960-12-31,1961-01-31,1961-02-28,1961-03-31,1961-04-30,1961-05-31,1961-06-30,1961-07-31,1961-08-31,1961-09-30,...,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30,2020-10-31,2020-11-30,2020-12-31
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,69.9000,69.9000,69.9000,69.9000,69.9000,69.9000,69.9000,69.9000,69.9000,69.6000
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.8000
2,696.3,696.3,696.3,696.3,696.3,696.3,696.3,696.3,696.3,696.3,...,660.0700,660.0700,660.0700,660.0700,660.0700,660.0700,660.0700,660.0700,660.0700,635.1200
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9.0000,9.0000,9.0000,9.0000,9.0000,9.0000,9.0000,9.0000,9.0000,9.0000
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-48079.2000,-48079.2000,-48079.2000,-48079.2000,-48079.2000,-48079.2000,-48079.2000,-48079.2000,-48079.2000,-105906.6000
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0000,2.0000,2.0000,2.0000,2.0000,2.0000,2.0000,2.0000,2.0000,-137.9000
1,696.3,696.3,696.3,696.3,696.3,696.3,696.3,696.3,696.3,696.3,...,660.0700,660.0700,660.0700,660.0700,660.0700,660.0700,660.0700,660.0700,660.0700,635.1200
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.0000,5.0000,5.0000,5.0000,5.0000,5.0000,5.0000,5.0000,5.0000,6.2500
3,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,15.9400,15.9400,15.9400,15.9400,15.9400,15.9400,15.9400,15.9400,15.9400,16.4200
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.8000


In [ ]:
df = df.fillna(value="NaN")

In [58]:
slice_ = df_resample.loc[:, :"1962-03-15"]
slice_

,1960-12-31,1961-01-31,1961-02-28,1961-03-31,1961-04-30,1961-05-31,1961-06-30,1961-07-31,1961-08-31,1961-09-30,1961-10-31,1961-11-30,1961-12-31,1962-01-31,1962-02-28
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,696.3,696.3,696.3,696.3,696.3,696.3,696.3,696.3,696.3,696.3,696.3,696.3,692.50,692.50,692.50
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,696.3,696.3,696.3,696.3,696.3,696.3,696.3,696.3,696.3,696.3,696.3,696.3,692.50,692.50,692.50
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.06,3.06,3.06
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [82]:
slice_.iloc[:, len(slice_.columns) - 5:].head(5)

,1961-10-31,1961-11-30,1961-12-31,1962-01-31,1962-02-28
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,696.3,696.3,692.5,692.5,692.5
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
